In [1]:
from thefuzz import process

In [5]:
import json

In [2]:
alt_labels = ["indigenous peoples", "aboriginal peoples", "traditional peoples", "native peoples", "indigenous culture"]

suggestions = ['Aboriginal',
  'Torres Strait Islander peoples',
  'First Nation peoples',
  'First Nation people',
  'First Nations people',
  'Indigenous',
  'Aboriginal peoples',
  'Aboriginal people']

In [3]:
for label in alt_labels:
    highest_score = process.extractOne(label, suggestions)
    
    if highest_score[1] == 100:
            print(f"{label}, exact_match: {highest_score}")
    if 100 > highest_score[1] >= 90:
           print(f"{label}, near_match: {highest_score}")

indigenous peoples, near_match: ('Indigenous', 90)
aboriginal peoples, exact_match: ('Aboriginal peoples', 100)
indigenous culture, near_match: ('Indigenous', 90)


In [20]:
label = "hhh"

In [21]:
process.extractOne(label, suggestions)

('Aboriginal', 0)

In [6]:
with open('/Users/anesterov/reps/LODlit/AAT/aat_query_results_en.json','r') as jf:
    aat_en = json.load(jf)

In [ ]:
# suggestion can be found in the same properties as query terms
# [{'query_term':'','entity_id':'','sug_value':'','found_in':'','close_match':(),'near_match':()}]

In [ ]:
def find_suggestions(resource:str, lang:str, query_results_path:str) -> dict:
    '''
    Finding suggestions from WM in the literal values of entities based on fuzzy string matching (thefuzz library);
    there can be an exact match (score = 100) or near match (score >= 90);
    
    resource: str, 'wikidata', 'aat', 'pwn', or 'odwn'
    lang: str, 'en' or 'nl'
    query_results_path: str, path to the file with query results of a resoure
    Returns dict per resource per lang; for example: {'query_term':
                    [{'query_term':'','entity_id':'',
                    'sug_value':'','found_in':'',
                    'exact_match':(),'near_match':()}]};
    '''
    
    results = {}
    
    # load suggestions and resource per lang
    if lang == "en":
        # change path
        with open('/Users/anesterov/reps/LODlit/RQ2/en_suggestions.json','r') as jf:
            en_suggestions = json.load(jf)
            
        if resource == "aat":
            with open(query_results_path,'r') as jf:
                aat_en = json.load(jf)
                
            for term, hits in aat_en.items():
                suggestions_found_per_term = []
                # if there are suggestions and hits for a query term
                if len(en_suggestions[term]) > 0 and len(hits) > 0:
                    # searhing suggestions in literals
                    for hit in hits:
                        
                        # prefLabel
                        match_prefLabel = process.extractOne(hit['prefLabel'], en_suggestions[term])
                        if match_prefLabel[1] == 100:
                            dict_per_hit['sug_value']
                            
                            # STOPPED HERE
                            
            print(f"{label}, exact_match: {highest_score}")
    if 100 > highest_score[1] >= 90:
           print(f"{label}, near_match: {highest_score}")
                    
            
    if lang == "nl":
        with open('/Users/anesterov/reps/LODlit/RQ2/nl_suggestions.json','r') as jf:
            nl_suggestions = json.load(jf)
    
        

In [34]:
# write the function for one term

def find_suggestions(resource:str, lang:str, term:str, hits:list) -> dict:
    '''
    '''
    dict_per_hit = {}
    list_of_found_suggestions = []
    
    # check if a query term has suggestions
    if len(suggestions.get(hit['query_term'])) == 0:
        dict_per_hit[hit['query_term']] = []
    else:
        # checking properties
        sug_hit = {}
        
        # prefLabel
        match_prefLabel = process.extractOne(hit['prefLabel'], suggestions[hit['query_term']])
        
        if 100 >= match_prefLabel[1] >= 90:
            sug_hit['entity_id'] = hit['aat_uri']
            sug_hit['sug_value'] = match_prefLabel[0]
            sug_hit['found_in'] = 'prefLabel'
            sug_hit['match_score'] = match_prefLabel[1]
            list_of_found_suggestions.append(sug_hit)

    
    dict_per_hit[hit['query_term']] = list_of_found_suggestions
    
    return dict_per_hit

In [49]:
match_score = 95

In [50]:
if 100 >= match_score >= 90:
    print(True)

True


In [ ]:
# subfunction for matches
def _get_match_type(match:tuple) -

In [41]:
suggestions = {'batavia':['smth', 'batavia']}

In [42]:
find_suggestions_in_hit(test_hit,suggestions)

{'batavia': [{'entity_id': '300012925',
   'sug_value': 'batavia',
   'found_in': 'prefLabel',
   'exact_match': None,
   'near_match': ('batavia', 90)}]}

In [23]:
test_hit = {}
for i,v in aat_en.items():
    if len(v) > 0:
        test_hit = v[0]
        break

In [29]:
test_hit

{'query_term': 'batavia',
 'aat_uri': '300012925',
 'found_in': 'prefLabel',
 'prefLabel': 'Batavia dammar',
 'prefLabel_comment': '',
 'altLabel': ['Batavia damar', 'dammar, Batavia'],
 'altLabel_comment': [],
 'scopeNote': 'General term referring to various dammar resins that were formerly shipped under the tradename "Batavia" from Jakarta (formerly Batavia), Indonesia. General term referring to various dammar resins that were formerly shipped under the tradename "Batavia" from Jakarta (formerly Batavia), Indonesia.'}

In [11]:
aat_en['batavia']

[{'query_term': 'batavia',
  'aat_uri': '300012925',
  'found_in': 'prefLabel',
  'prefLabel': 'Batavia dammar',
  'prefLabel_comment': '',
  'altLabel': ['Batavia damar', 'dammar, Batavia'],
  'altLabel_comment': [],
  'scopeNote': 'General term referring to various dammar resins that were formerly shipped under the tradename "Batavia" from Jakarta (formerly Batavia), Indonesia. General term referring to various dammar resins that were formerly shipped under the tradename "Batavia" from Jakarta (formerly Batavia), Indonesia.'},
 {'query_term': 'batavia',
  'aat_uri': '300012925',
  'found_in': 'altLabel',
  'prefLabel': 'Batavia dammar',
  'prefLabel_comment': '',
  'altLabel': ['Batavia damar', 'dammar, Batavia'],
  'altLabel_comment': [],
  'scopeNote': 'General term referring to various dammar resins that were formerly shipped under the tradename "Batavia" from Jakarta (formerly Batavia), Indonesia. General term referring to various dammar resins that were formerly shipped under th

In [12]:
with open('/Users/anesterov/reps/LODlit/RQ2/en_suggestions.json','r') as jf:
    en_suggestions = json.load(jf)

In [15]:
en_suggestions['aboriginal']

['Indigenous',
 'First Nation people',
 'First Nation peoples',
 'First Nations people',
 'Aboriginal',
 'Torres Strait Islander peoples',
 'Aboriginal peoples',
 'Aboriginal people']